In [12]:
# import
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

import time
from tqdm import tqdm_notebook

from bs4 import BeautifulSoup

In [13]:
# 홈페이지를 selenium으로 열기
driver = webdriver.Chrome()
driver.get('https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix/TV-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8?genres=drm&release_year_from=2020&release_year_until=2023&sort_by=release_year')


# 팝업창 닫기
try:
    driver.find_element(By.CSS_SELECTOR, '#uc-center-container > div.sc-eBMEME.hasieq > div > div > div > button.sc-dcJsrY.gDlDbR').click()
except NoSuchElementException:
    pass

time.sleep(1)

# 현재 페이지 높이 가져오기
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 새로운 콘텐츠가 로드되기를 기다림
    time.sleep(2)
    
    # 새로운 페이지 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 새로운 높이가 이전 높이와 같으면, 더 이상 로드할 콘텐츠가 없다는 의미
    if new_height == last_height:
        break
    
    last_height = new_height

# 현재 페이지 내의 정보를 저장
raw = driver.page_source

# BeautifulSoup을 이용하여 html 파싱
soup = BeautifulSoup(raw, 'html.parser')

driver.close()

# url_list 만들기
base_url = 'https://www.justwatch.com'
url_list = []

# 타이틀 리스트 내의 URL을 추출
for i in tqdm_notebook(range(2, len(soup.select('.title-list-grid')[0]) + 1)):
    tmp = soup.select_one('#base > div.title-list.title-list--CLS-block > div:nth-child(2) > div > div > div:nth-child(1) > div > div:nth-child(' + str(i) + ') > a')
    if tmp:
        tmp_url = tmp.get('href')
        if tmp_url:
            url = base_url + tmp_url
            url_list.append(url)

C:\Users\rbfl7\AppData\Local\Temp\ipykernel_30976\2990245862.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(2, len(soup.select('.title-list-grid')[0]) + 1)):


  0%|          | 0/721 [00:00<?, ?it/s]

In [14]:
# 개수 확인
len(soup.select('.title-list-grid')[0])

722

In [15]:
# 개수 확인
len(url_list)

721

In [16]:
# 테스트
movie_list = []

for x in url_list:
    driver = webdriver.Chrome()
    driver.get(x)
    
    # 팝업창 닫기
    try:
        driver.find_element(By.CSS_SELECTOR, '#uc-center-container > div.sc-eBMEME.hasieq > div > div > div > button.sc-dcJsrY.gDlDbR').click()
    except NoSuchElementException:
        pass

    time.sleep(2)

    raw = driver.page_source
    soup = BeautifulSoup(raw, 'html.parser')

    # 'detail-infos' 클래스의 모든 'div' 태그를 찾습니다.
    detail_infos = soup.find_all('div', class_='detail-infos')

    # 중복을 제거하기 위해 집합(set)을 사용합니다.
    unique_detail_infos = list({str(info): info for info in detail_infos}.values())

    # 정보를 저장할 변수 초기화
    play_time_div = None
    genre_div = None
    rating_div = None
    country_div = None
    director_div = None

    # 필요한 정보를 가진 'div' 태그를 각각 찾습니다.
    for info in unique_detail_infos:
        heading = info.find('h3', class_='detail-infos__subheading')
        if heading:
            if '재생 시간' in heading.text:
                play_time_div = info.find('div', class_='detail-infos__value')
            elif '장르' in heading.text:
                genre_div = info.find('div', class_='detail-infos__value')
            elif '연령 등급' in heading.text:
                rating_div = info.find('div', class_='detail-infos__value')
            elif 'Production country' in heading.text:
                country_div = info.find('div', class_='detail-infos__value')
            elif '감독' in heading.text:
                director_div = info.find('div', class_='detail-infos__value')

    # 정보를 텍스트로 가져옵니다.
    play_time = play_time_div.text.strip() if play_time_div else None
    genre = genre_div.text.strip() if genre_div and genre_div.text.strip() else None
    rating = rating_div.text.strip() if rating_div else None
    country = country_div.text.strip() if country_div else None
    director = director_div.text.strip() if director_div else None

    # 추가 정보 추출
    title_block = soup.find('div', class_='title-block')
    korean_title_element = title_block.find('h1')
    original_title_element = title_block.find('h3')
    seasons_element = soup.find('h2', class_='subheading')

    korean_title = korean_title_element.text.strip() if korean_title_element else None
    original_title = original_title_element.text[5:].strip() if original_title_element else None
    seasons = seasons_element.text.strip() if seasons_element else None

    # 영화 데이터 저장
    movie_data = {
        "한글 이름": korean_title,
        "원제": original_title,
        "시즌 수": seasons,
        "재생 시간": play_time,
        "장르": genre,
        "연령 등급": rating,
        "Production country": country,
        "감독": director
    }
    
    movie_list.append(movie_data)

# DataFrame 생성
jw_df = pd.DataFrame(movie_list)

# 드라이버 종료
driver.quit()

In [17]:
# 저장
jw_df.to_csv('../data/justwatch.csv', sep=",", encoding="utf-8")

jw_df.to_excel('../data/justwatch.xlsx')